# LOADING LIBRARIES

In [1]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score


# LOADING PATIENT DATA SET

In [26]:
#regression
df = pd.read_csv(r'droppedDatasets/trainingSet.csv')

In [27]:
#scalled binary
df_test = pd.read_csv(r'droppedDatasets/testingSet.csv')

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4084 entries, 0 to 4083
Data columns (total 41 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   ID                           4084 non-null   int64  
 1   Age                          4084 non-null   int64  
 2   Race                         4084 non-null   object 
 3   Sex                          4084 non-null   object 
 4   State                        4084 non-null   object 
 5   Zip Code                     4084 non-null   object 
 6   MSA                          4084 non-null   object 
 7   Enrollment Type Categorized  4084 non-null   int64  
 8   Enrollment Months            4084 non-null   float64
 9   HCC Score                    4071 non-null   float64
 10  Avg. LOS                     4084 non-null   float64
 11  Diagnosis                    4084 non-null   int64  
 12  # Total Claims               4084 non-null   int64  
 13  # Hospital OP     

In [2]:
df_test = pd.read_csv(r'cleanedDatasets\cleanedFeatures.csv')

In [ ]:
df_test.info()

In [162]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16328 entries, 0 to 16327
Data columns (total 42 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   ID                           16328 non-null  float64
 1   Age                          16328 non-null  float64
 2   Zip Code                     16328 non-null  float64
 3   Enrollment Type Categorized  16328 non-null  float64
 4   Enrollment Months            16328 non-null  float64
 5   HCC Score                    16328 non-null  float64
 6   Avg. LOS                     16328 non-null  float64
 7   Diagnosis                    16328 non-null  float64
 8   # Total Claims               16328 non-null  float64
 9   # Hospital OP                16328 non-null  float64
 10  # All Physician OP           16328 non-null  float64
 11  # Inpatient                  16328 non-null  float64
 12  # Short Term Stay Hospital   16328 non-null  float64
 13  # Long Term Stay

# CLEANING NULLS

In [5]:
df['# Inpatient'].fillna(0.0,inplace=True)
df['HCC Score'].fillna(df['HCC Score'].mean(),inplace=True)
df['# Readmissions'].fillna(0.0,inplace=True)
df['# Part B E&M'].fillna(0.0,inplace=True)
df['# Part B Labs'].fillna(4.0,inplace=True)
df['# Part-B Dialysis'].fillna(0.0,inplace=True)

# CATAGORIZING DATA

In [7]:
# df['Y'] = np.where(df['TotalCost_Y_Actual'] > 11000, 1, 0)
df['HCC Score'] = np.where(df['HCC Score'] < 1, 0, 1)
df['Age'] = np.where(df['Age'] < 64, 0, 1)
feature_names = df.columns.values.tolist()
for feature_name in feature_names:
    if '#' == feature_name[0]: 
        print(feature_name)
        df[feature_name] = np.where( ((df[feature_name] > 3) & (df[feature_name] <= 6)), 3, df[feature_name])
        df[feature_name] = np.where( ( (df[feature_name] > 6) & (df[feature_name] <= 13) ), 5,df[feature_name])
        df[feature_name] = np.where(df[feature_name] > 13, 7,df[feature_name])

# df['Y'] = df['TotalCost_Y_Actual']

# Total Claims
# Hospital OP
# All Physician OP
# Inpatient
# Short Term Stay Hospital
# Long Term Stay Hospital
# Rehabilitation Hospital
# Psychiatric Hospital
# Readmissions
# ER
# ER Admissions
# SNF
# Non Swing Bed SNF Claim
# Swing Bed SNF Claim
# Home Health
# Hospice
# Labs
# Part B Labs
# Imaging
# Part B Imaging
# Part B E&M
# Part B Drugs
# Part B Ambulance
# Dialysis
# Part-B Dialysis
# Rx Claims
# DME
# Miscellaneous


# LABEL ENCODING

In [10]:
df_placeholder = df
from sklearn.preprocessing import LabelEncoder
categorical_cols = df_placeholder.select_dtypes(include='object').columns

# Initialize LabelEncoder
label_encoder = LabelEncoder()

# Apply label encoding to all categorical columns
for col in categorical_cols:
    df_placeholder[col + '_encoded'] = label_encoder.fit_transform(df_placeholder[col])

# Drop original categorical columns
df_placeholder.drop(categorical_cols, axis=1, inplace=True)
df = df_placeholder

# REMOVING NUMERICAL LABELS

In [ ]:
df.drop(['TotalCost_Y_Actual','TotalCost_Y_Expected'],axis=1,inplace=True)

# OVERSAMPLING 


In [30]:
from imblearn.over_sampling import SMOTE
import pandas as pd

# Define the SMOTE object
smote = SMOTE(sampling_strategy='minority', k_neighbors=5, random_state=42)

# Apply SMOTE to the training set
X_train_resampled, y_train_resampled = smote.fit_resample(df.drop('Y',axis=1), df['Y'])
# Instantiate the oversampler and fit the data
df = X_train_resampled
df['Y'] = y_train_resampled


Resampled data shape: (24400, 35) (24400,)


# NORMALIZATION

In [173]:

from sklearn.preprocessing import MinMaxScaler
# initialize MinMaxScaler
scaler = MinMaxScaler()
label = df["Y"]
df.drop('Y',axis=1,inplace=True)
# fit and transform the data
scaled_data = scaler.fit_transform(df)

# create a new dataframe with the scaled data
df = pd.DataFrame(scaled_data, columns=df.columns)
df['Y'] = label


# SPLIT

In [31]:

# Separate the target variable from the features
target = df['Y']
# features = df[feature_names]
# features = df.drop(['Y',"ID","Zip Code",'MSA_encoded'],axis=1)
# features.drop(dropped_features,axis=1,inplace=True)
# features = df[labels]
features = df.drop('Y',axis=1)
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.31, random_state=42)

# FEATURE IMPORTANCE FROM RF

In [ ]:

rf = RandomForestClassifier(n_estimators=100)

# Fit the classifier on the training set
rf.fit(X_train, y_train)

# Get the feature importances
importances = rf.feature_importances_

# Create a dictionary of feature names and importances
feature_importances = dict(zip(features.columns, importances))

# Sort the dictionary by feature importance (descending order)
sorted_importances = sorted(feature_importances.items(), key=lambda x: x[1], reverse=True)

# Print the sorted list of feature names and importances
n = 0
for feature, importance in sorted_importances:
    
    print(f"{n} {feature}: {importance}")
    n+=1
# Use the trained model to predict on the test set
y_pred = rf.predict(X_test)


# Generate a confusion matrix and calculate the metrics
cm = confusion_matrix(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

# Print the confusion matrix and metrics
print("Confusion matrix:")
print(pd.crosstab(y_test, y_pred, rownames=['True'], colnames=['Predicted'], margins=True))
print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")

In [181]:
df.drop(['% Readmissions','# Readmissions','# Rehabilitation Hospital',"# Hospice",'# Psychiatric Hospital','# Psychiatric Hospital','# Total Claims','# Swing Bed SNF Claim','# Long Term Stay Hospital','# ER Admissions','# SNF','# Non Swing Bed SNF Claim','# Miscellaneous'],axis=1,inplace=True)

# TEST SET PREDICTION

In [ ]:
df_test.info()

In [68]:
df_test = df_test[labels]

In [109]:
df_test.drop(['ID'],axis=1,inplace=True)
# df_test.drop(['% Readmissions','# Readmissions','# Rehabilitation Hospital',"# Hospice",'# Psychiatric Hospital','# Psychiatric Hospital','# Total Claims','# Swing Bed SNF Claim','# Long Term Stay Hospital','# ER Admissions','# SNF','# Non Swing Bed SNF Claim','# Miscellaneous'],axis=1,inplace=True)

In [110]:
df_test.drop('MSA_encoded',axis=1,inplace=True)

In [111]:
labels = clf.predict(df_test)

# NOMRALIZING TESTING FEATURES

In [11]:

from sklearn.preprocessing import MinMaxScaler
# initialize MinMaxScaler
scaler = MinMaxScaler()
# fit and transform the data
scaled_data = scaler.fit_transform(df_test)
# create a new dataframe with the scaled data
df_test = pd.DataFrame(scaled_data, columns=df_test.columns)


In [37]:
df_pred = pd.read_csv(r'high-cost-diabetes-patients-prediction/Softec_Patient_Data_Testing_Kaggle_V1.csv')

In [ ]:

import tensorflow as tf
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.preprocessing import LabelEncoder
from tensorflow import keras
y = y_train
X = X_train

model = Sequential()
model.add(Dense(128, input_dim=X.shape[1], activation='relu'))
model.add(Dense(128, input_dim=X.shape[1], activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(loss='binary_crossentropy', optimizer=keras.optimizers.Adam(0.0001), metrics=['accuracy'])
def schedule(epoch, lr):    
    if epoch > 5:
        return lr * tf.math.exp(-0.01)
    else:
        return lr
model_learningRate_schedular = tf.keras.callbacks.LearningRateScheduler(schedule, verbose=1)
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor="val_accuracy",
    min_delta=0.05,
    patience=20,
    mode="auto",
    restore_best_weights=True,
)
my_callbacks = [model_learningRate_schedular, early_stopping]
# Fit the model to the data
model.fit(X, y, epochs=100, batch_size=128, validation_data=(X_test,y_test),callbacks=my_callbacks)

# Get the feature importance
weights = model.layers[0].get_weights()[0]
feature_importance = np.sum(weights, axis=1)

# Rank the features from top to bottom
feature_importance_df = pd.DataFrame({'Feature': X.columns, 'Importance': feature_importance})
feature_importance_df = feature_importance_df.sort_values('Importance', ascending=False).reset_index(drop=True)

# Print the feature importance rankings
print("Feature Importance:")
print(feature_importance_df)


In [50]:
labels = model.predict(df_test)

128/128 [==============================] - 0s 998us/step


In [51]:
labels = labels > 0.5

In [112]:
df_pred['Prediction'] = labels

In [113]:
df_pred['Prediction'].value_counts()

0    3335
1     749
Name: Prediction, dtype: int64

In [43]:
df_pred.drop('prediction',axis=1,inplace=True)

In [139]:
df_pred.to_csv(r'predictions.csv',index=False)

# REGRESSION

In [97]:
y_train.values.shape

(11429,)

In [20]:

import tensorflow as tf
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.preprocessing import LabelEncoder
from tensorflow import keras
y = y_train
X = X_train

model = Sequential()
model.add(Dense(128, input_dim=X.shape[1], activation='relu'))
model.add(Dense(128, input_dim=X.shape[1], activation='relu'))
model.add(Dense(1))

# Compile the model
model.compile(loss='mean_squared_error', optimizer=keras.optimizers.Adam(0.0001), metrics = ["mean_squared_error"])
def schedule(epoch, lr):    
    if epoch > 5:
        return lr * tf.math.exp(-0.01)
    else:
        return lr
model_learningRate_schedular = tf.keras.callbacks.LearningRateScheduler(schedule, verbose=1)
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor="mean_squared_error",
    min_delta=0.05,
    patience=20,
    mode="auto",
    restore_best_weights=True,
)
my_callbacks = [model_learningRate_schedular, early_stopping]
# Fit the model to the data
model.fit(X.values, y.values, epochs=500, batch_size=256, validation_data=(X_test,y_test),callbacks=my_callbacks)

# Get the feature importance
weights = model.layers[0].get_weights()[0]
feature_importance = np.sum(weights, axis=1)

# Rank the features from top to bottom
feature_importance_df = pd.DataFrame({'Feature': X.columns, 'Importance': feature_importance})
feature_importance_df = feature_importance_df.sort_values('Importance', ascending=False).reset_index(drop=True)

# Print the feature importance rankings
print("Feature Importance:")
print(feature_importance_df)



Epoch 1: LearningRateScheduler setting learning rate to 9.999999747378752e-05.
Epoch 1/500
45/45 [==============================] - 1s 7ms/step - loss: 649200960.0000 - mean_squared_error: 649200960.0000 - val_loss: 701773376.0000 - val_mean_squared_error: 701773376.0000 - lr: 1.0000e-04

Epoch 2: LearningRateScheduler setting learning rate to 9.999999747378752e-05.
Epoch 2/500
45/45 [==============================] - 0s 5ms/step - loss: 649181184.0000 - mean_squared_error: 649181184.0000 - val_loss: 701749248.0000 - val_mean_squared_error: 701749248.0000 - lr: 1.0000e-04

Epoch 3: LearningRateScheduler setting learning rate to 9.999999747378752e-05.
Epoch 3/500
45/45 [==============================] - 0s 5ms/step - loss: 649151744.0000 - mean_squared_error: 649151744.0000 - val_loss: 701710464.0000 - val_mean_squared_error: 701710464.0000 - lr: 1.0000e-04

Epoch 4: LearningRateScheduler setting learning rate to 9.999999747378752e-05.
Epoch 4/500
45/45 [==============================]

# SVC

In [122]:
# Import necessary libraries
import pandas as pd
from sklearn import svm
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt



# Create a SVM classifier with a linear kernel
clf = svm.SVC(kernel='linear')

# Fit the model to the training data
clf.fit(X_train, y_train)

# Make predictions on the testing data
y_pred = clf.predict(X_test)


# Generate a confusion matrix and calculate the metrics
cm = confusion_matrix(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

# Print the confusion matrix and metrics
print("Confusion matrix:")
print(pd.crosstab(y_test, y_pred, rownames=['True'], colnames=['Predicted'], margins=True))
print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")

Confusion matrix:
Predicted    0  1  All
True                  
0          392  0  392
1          114  1  115
All        506  1  507
Accuracy: 0.7751479289940828
Precision: 0.8258063007226886
Recall: 0.7751479289940828
F1 Score: 0.678932620905056


In [34]:
import pandas as pd
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC


# Initialize the LinearSVC model with L1 penalty and squared hinge loss
model_l1 = LinearSVC(penalty="l1", loss="squared_hinge", dual=False, tol=1e-3)

# Use SelectFromModel to automatically select the most important features
feature_selector = SelectFromModel(model_l1)


# Train the model on the selected features
model_l1.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model_l1.predict(X_test)

# Compute the confusion matrix
cm = confusion_matrix(y_test, y_pred)
print(f"Confusion matrix:\n{cm}")

# Compute precision, recall, and F1 score
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 score: {f1}")



Confusion matrix:
[[2615 1183]
 [1387 2379]]
Precision: 0.6678832116788321
Recall: 0.6317047265002655
F1 score: 0.6492903930131004


c:\Users\muham\miniconda3\envs\tf\lib\site-packages\sklearn\svm\_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [129]:
X_train_selected.shape

(15821, 33)

In [38]:
labels = model_l1.predict(df_test)

In [39]:
df_pred['Prediction'] = labels

In [40]:
df_pred['Prediction'].value_counts()

1    3298
0     786
Name: Prediction, dtype: int64

In [159]:
df_test.drop(dropped_features,axis=1,inplace=True)

# ENSEMBLE

In [ ]:
def majority_voting(x, y, z):
    """
    Performs majority voting on the predictions of three models
    
    Args:
    pred1 (array-like): predicted class labels of model1
    pred2 (array-like): predicted class labels of model2
    pred3 (array-like): predicted class labels of model3
    
    Returns:
    array-like: predicted class labels by majority voting
    """
    ensemble_pred = []
    for i in range(len(x)):
        # count the occurrence of each class label
        count = {0: 0, 1: 0} # assume we have three classes
        count[x[i]] += 1
        count[y[i]] += 1
        count[z[i]] += 1
        
        # find the class label with the highest count
        majority_label = max(count, key=count.get)
        ensemble_pred.append(majority_label)
        
    return ensemble_pred
x = model.predict(df_test)

y = rf.predict(df_test)
z = model_l1.predict(df_test)

In [150]:
x = x >= 0.5
a = pd.DataFrame(x)

In [154]:
a[0].value_counts()

False    507
Name: 0, dtype: int64

In [14]:
# df.drop(dropped_features,axis=1,inplace=True)
df.drop(["ID","Zip Code_encoded",'MSA_encoded'],axis=1,inplace=True)


In [12]:
df_test.drop(dropped_features,axis=1,inplace=True)

NameError: name 'dropped_features' is not defined

In [179]:
df_test.drop(['ID','MSA_encoded'],axis=1,inplace=True)

In [180]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4084 entries, 0 to 4083
Data columns (total 34 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Age                          4084 non-null   int64  
 1   Enrollment Type Categorized  4084 non-null   int64  
 2   Enrollment Months            4084 non-null   float64
 3   HCC Score                    4084 non-null   int64  
 4   Avg. LOS                     4084 non-null   float64
 5   Diagnosis                    4084 non-null   int64  
 6   # Total Claims               4084 non-null   int64  
 7   # Hospital OP                4084 non-null   int64  
 8   # All Physician OP           4084 non-null   int64  
 9   # Inpatient                  4084 non-null   int64  
 10  # Short Term Stay Hospital   4084 non-null   int64  
 11  # Rehabilitation Hospital    4084 non-null   int64  
 12  # Psychiatric Hospital       4084 non-null   int64  
 13  % Readmissions    

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4084 entries, 0 to 4083
Data columns (total 38 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Age                          4084 non-null   int32  
 1   Enrollment Type Categorized  4084 non-null   int64  
 2   Enrollment Months            4084 non-null   float64
 3   HCC Score                    4084 non-null   int32  
 4   Avg. LOS                     4084 non-null   float64
 5   Diagnosis                    4084 non-null   int64  
 6   # Total Claims               4084 non-null   int64  
 7   # Hospital OP                4084 non-null   int64  
 8   # All Physician OP           4084 non-null   int64  
 9   # Inpatient                  4084 non-null   float64
 10  # Short Term Stay Hospital   4084 non-null   int64  
 11  # Long Term Stay Hospital    4084 non-null   int64  
 12  # Rehabilitation Hospital    4084 non-null   int64  
 13  # Psychiatric Hosp

In [41]:
df_pred.to_csv(r'Predictions_over_sampled_linearSVC.csv',index=False)


In [185]:
df_test.to_csv(r'droppedDatasets/testingSet.csv')